In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import warnings
warnings.filterwarnings('ignore')

### Survey patients characteristics

Download the dataset from https://www.kaggle.com/new-york-state/nys-patient-characteristics-survey-pcs-2015 

In [2]:
file_path = '../data/gitignoreDATA/nys-patient-characteristics-survey-pcs-2015/patient-characteristics-survey-pcs-2015.csv'

check_df = pd.read_csv(file_path)
print(f'shape: {check_df.shape}')
n_cols_unknown = len([col for col in check_df.columns if "UNKNOWN" in check_df[col].values])

print(f'{n_cols_unknown} out of {len(check_df.columns)} columns contain the UNKNOWN value:\n')

for col in check_df.columns:
    col_vals = check_df[col].values
    if 'UNKNOWN' in col_vals:
        perc_unknown = len(col_vals[col_vals=='UNKNOWN'])/len(col_vals)
        print(f"{col:<30} : {round(perc_unknown,3):>5} % of unknown values")
        if perc_unknown>0.1:
            #rimuovi la colonna
            check_df.drop(col,1,inplace=True)
            print(f'removing column {col}')
        else:
            #rimuovi le righe
            indexes_unknown = check_df[check_df[col]=="UNKNOWN"].index.values
            check_df.drop(indexes_unknown,inplace=True)
            print(f'removing {len(indexes_unknown)} instances with unknown value in {col}\n')


shape: (179096, 67)
58 out of 67 columns contain the UNKNOWN value:

Age Group                      :   0.0 % of unknown values
removing 18 instances with unknown value in Age Group

Sex                            : 0.002 % of unknown values
removing 376 instances with unknown value in Sex

Transgender                    : 0.027 % of unknown values
removing 4865 instances with unknown value in Transgender

Sexual Orientation             : 0.077 % of unknown values
removing 13441 instances with unknown value in Sexual Orientation

Hispanic Ethnicity             : 0.015 % of unknown values
removing 2336 instances with unknown value in Hispanic Ethnicity

Living Situation               : 0.008 % of unknown values
removing 1208 instances with unknown value in Living Situation

Household Composition          : 0.019 % of unknown values
removing 2912 instances with unknown value in Household Composition

Preferred Language             : 0.002 % of unknown values
removing 369 instances with u

In [3]:
mental_illness_related = ['Mental Illness','Serious Mental Illness','Other Developmental Disability','Alcohol Related Disorder',
       'Drug Substance Disorder']
labels_df = check_df.groupby(mental_illness_related).size().reset_index()

#we delete all the instances with labels that do not appear at least 50 times
labels_to_drop = labels_df[labels_df[0]<50]
labels_to_drop

,Mental Illness,Serious Mental Illness,Other Developmental Disability,Alcohol Related Disorder,Drug Substance Disorder,0
5,NO,NO,YES,NO,YES,2
6,NO,NO,YES,YES,NO,8
7,NO,NO,YES,YES,YES,5
9,NO,YES,NO,NO,YES,30
10,NO,YES,NO,YES,NO,19
11,NO,YES,NO,YES,YES,18
13,NO,YES,YES,NO,YES,1
14,NO,YES,YES,YES,YES,3
20,YES,NO,YES,NO,YES,15
21,YES,NO,YES,YES,NO,10


In [4]:
#drop the rows with these classifications
x_cols = set(labels_to_drop.columns.tolist())
df_cols = set(check_df.columns.tolist())
common_cols = list(x_cols & df_cols)

rows_to_drop = labels_to_drop.merge(check_df.reset_index(), on=common_cols)['index'].values.tolist()
print(rows_to_drop)

check_df = check_df.drop(rows_to_drop).reset_index(drop=True)

[19699, 137996, 1638, 1712, 7399, 20487, 36666, 72888, 109201, 127466, 1684, 29582, 37984, 39731, 56431, 9751, 27081, 27401, 27648, 28421, 28605, 29843, 30101, 30650, 30880, 41914, 42378, 52078, 58681, 64684, 65723, 66495, 75084, 76153, 80943, 81529, 82204, 106655, 118314, 119305, 131627, 133232, 133666, 137124, 165356, 2115, 25227, 28945, 41568, 43941, 44392, 55059, 55670, 59676, 73936, 78529, 103489, 120293, 129415, 129452, 129922, 132645, 133655, 158949, 20057, 25262, 26337, 28422, 31991, 36945, 49248, 78759, 92424, 129028, 129088, 129466, 137126, 140241, 149625, 159768, 163735, 164137, 30045, 53962, 76769, 100592, 13333, 13776, 14021, 25846, 28163, 29193, 30028, 58295, 59932, 60967, 72036, 72715, 73188, 124421, 129891, 15545, 41755, 44550, 59849, 72828, 72846, 81797, 83073, 95369, 162227, 14036, 14595, 27074, 28822, 31957, 97952, 178431]


In [5]:
binary_cols = []
cols_2drop = []
for col in check_df.columns.values:
    grouped = check_df.groupby(col).size()
    print(grouped)
    if len(grouped)==2:
        binary_cols.append(col)
    if len(grouped)==1:
        cols_2drop.append(col)
        print(f'Drop column {col} because there is just one value for all instances')
    print()

Survey Year
2015    105098
dtype: int64
Drop column Survey Year because there is just one value for all instances

Program Category
EMERGENCY       1179
INPATIENT       5662
OUTPATIENT     72465
RESIDENTIAL    18334
SUPPORT         7458
dtype: int64

Region Served
CENTRAL NY REGION       11027
HUDSON RIVER REGION     13683
LONG ISLAND REGION       8544
NEW YORK CITY REGION    57053
WESTERN REGION          14791
dtype: int64

Age Group
ADULT    87977
CHILD    17121
dtype: int64

Sex
FEMALE    55121
MALE      49977
dtype: int64

Transgender
CLIENT DIDN'T ANSWER       471
NO, NOT TRANSGENDER     103871
YES, TRANSGENDER           756
dtype: int64

Sexual Orientation
BISEXUAL                     2716
CLIENT DID NOT ANSWER        3144
LESBIAN OR GAY               3304
OTHER                         701
STRAIGHT OR HETEROSEXUAL    95233
dtype: int64

Hispanic Ethnicity
NO, NOT HISPANIC/LATINO    79498
YES                        25600
dtype: int64

Race
BLACK ONLY      26989
MULTI-RACIAL     29

In [6]:
binary_cols

['Age Group',
 'Sex',
 'Hispanic Ethnicity',
 'Veteran Status',
 'Mental Illness',
 'Intellectual Disability',
 'Autism Spectrum',
 'Other Developmental Disability',
 'Alcohol Related Disorder',
 'Drug Substance Disorder',
 'Mobility Impairment Disorder',
 'Hearing Visual Impairment',
 'Hyperlipidemia',
 'High Blood Pressure',
 'Diabetes',
 'Obesity',
 'Heart Attack',
 'Stroke',
 'Other Cardiac',
 'Pulmonary Asthma',
 'Alzheimer or Dementia',
 'Kidney Disease',
 'Liver Disease',
 'Endocrine Condition',
 'Neurological Condition',
 'Traumatic Brain Injury',
 'Joint Disease',
 'Cancer',
 'Other Chronic Med Condition',
 'No Chronic Med Condition',
 'Smokes',
 'Received Smoking Medication',
 'Received Smoking Counseling',
 'Serious Mental Illness',
 'SSI Cash Assistance',
 'SSDI Cash Assistance',
 'Veterans Disability Benefits',
 'Veterans Cash Assistance',
 'Public Assistance Cash Program',
 'Other Cash Benefits',
 'Medicaid and Medicare Insurance',
 'No Insurance',
 'Medicaid Insurance',


In [7]:
cols_2drop
check_df.drop(cols_2drop,1,inplace=True)
# we also have to remove the information on the Principal Diagnosis Class
check_df.drop('Principal Diagnosis Class',1,inplace=True)
check_df.reset_index(drop=True,inplace=True)

In [8]:
dummified_binary_df = pd.get_dummies(check_df[binary_cols],drop_first=True)
dummified_binary_df_rename = dummified_binary_df.drop(['Age Group_CHILD','Sex_MALE'],1)
dummified_binary_df_rename.columns = binary_cols[2:]
dummified_binary_df = pd.concat([dummified_binary_df[['Age Group_CHILD','Sex_MALE']],dummified_binary_df_rename],1)
dummified_binary_df.head()

,Age Group_CHILD,Sex_MALE,Hispanic Ethnicity,Veteran Status,Mental Illness,Intellectual Disability,Autism Spectrum,Other Developmental Disability,Alcohol Related Disorder,Drug Substance Disorder,...,Public Assistance Cash Program,Other Cash Benefits,Medicaid and Medicare Insurance,No Insurance,Medicaid Insurance,Medicare Insurance,Private Insurance,Child Health Plus Insurance,Other Insurance,Criminal Justice Status
0,0,1,1,0,1,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,0,1,0,0,1,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,1
4,0,1,0,1,1,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1


In [9]:
dummified_nonbinary_df = pd.get_dummies(check_df.drop(binary_cols,1),drop_first=False)
dummified_nonbinary_df.head()

,Three Digit Residence Zip Code,Program Category_EMERGENCY,Program Category_INPATIENT,Program Category_OUTPATIENT,Program Category_RESIDENTIAL,Program Category_SUPPORT,Region Served_CENTRAL NY REGION,Region Served_HUDSON RIVER REGION,Region Served_LONG ISLAND REGION,Region Served_NEW YORK CITY REGION,...,Education Status_NO FORMAL EDUCATION,Education Status_OTHER,Education Status_PRE-K TO FIFTH GRADE,Education Status_SOME COLLEGE,Special Education Services_NO,Special Education Services_NOT APPLICABLE,Special Education Services_YES,Medicaid Managed Insurance_NO,Medicaid Managed Insurance_NOT APPLICABLE,Medicaid Managed Insurance_YES
0,113,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
1,107,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
2,888,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
3,113,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
4,130,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0


In [10]:
dummified_check_df = pd.concat([dummified_binary_df,dummified_nonbinary_df],1)

In [11]:
dummified_check_df.rename(columns={'Three Digit Residence Zip Code':'ZIP3'},inplace=True)
dummified_ZIP = pd.get_dummies(dummified_check_df['ZIP3'],drop_first=False,prefix_sep='=',prefix='ZIP3')
dummified_check_df.drop('ZIP3',1,inplace=True)
dummified_check_df = pd.concat([dummified_check_df,dummified_ZIP],1)

In [12]:
dummified_check_df.head()

,Age Group_CHILD,Sex_MALE,Hispanic Ethnicity,Veteran Status,Mental Illness,Intellectual Disability,Autism Spectrum,Other Developmental Disability,Alcohol Related Disorder,Drug Substance Disorder,...,ZIP3=142,ZIP3=143,ZIP3=144,ZIP3=145,ZIP3=146,ZIP3=147,ZIP3=148,ZIP3=149,ZIP3=888,ZIP3=999
0,0,1,1,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,1,0,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
dummified_check_df.to_csv('../data/mental_illness.csv',index=True)

In [14]:
mental_illness_related

['Mental Illness',
 'Serious Mental Illness',
 'Other Developmental Disability',
 'Alcohol Related Disorder',
 'Drug Substance Disorder']

In [15]:
numerical_vars = []
labels_name = mental_illness_related
categorical_vars = [col for col in dummified_check_df if col not in labels_name]

np.save('../data/numerical_mental_vars.npy',numerical_vars)
np.save('../data/labels_name_mental.npy',labels_name)
np.save('../data/categorical_mental_vars.npy',categorical_vars)